<a href="https://colab.research.google.com/github/NathalyDM/genetifics/blob/main/ImageProcessingTutorials/SegmentationTutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **1. Import Libraries**


In [ ]:
# Import Necessary Libraries
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt
import albumentations as A
from sklearn.model_selection import train_test_split


## **2. Load Data Paths**


In [ ]:
# Provided Paths
image_paths = ['path_to_image1', 'path_to_image2', ...]  # Replace with actual paths
mask_paths = ['path_to_mask1', 'path_to_mask2', ...]  # Replace with actual paths

## **3. Split Data into Training and Validation Sets**

In [ ]:
# Split into Training and Testing Sets
train_image_paths, val_image_paths, train_mask_paths, val_mask_paths = train_test_split(image_paths, mask_paths, test_size=0.2, random_state=42)

## **4. Define Preprocessing Functions**

In [ ]:
# Load and Preprocess
def load_and_preprocess(image_path, mask_path, augment=True):
    image = cv2.imread(image_path)
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
    image = image / 255.0
    mask = mask / 255.0

    if augment:
        augmenter = A.Compose([A.HorizontalFlip(p=0.5), A.VerticalFlip(p=0.5), A.Rotate(limit=90, p=0.5)])
        augmented = augmenter(image=image, mask=mask)
        image = augmented['image']
        mask = augmented['mask']

    mask = np.expand_dims(mask, axis=-1)
    return image, mask

# Data Generator
def data_generator(image_paths, mask_paths, batch_size=32, augment=True):
    while True:
        idxs = np.arange(len(image_paths))
        np.random.shuffle(idxs)

        for start in range(0, len(image_paths), batch_size):
            end = min(start + batch_size, len(image_paths))
            batch_idxs = idxs[start:end]

            batch_images = []
            batch_masks = []

            for i in batch_idxs:
                image, mask = load_and_preprocess(image_paths[i], mask_paths[i], augment)
                batch_images.append(image)
                batch_masks.append(mask)

            yield np.array(batch_images), np.array(batch_masks)


## **5. Create Model**

In [ ]:
def create_segmentation_model(base_model_name, input_shape):
    base_model_class = getattr(tf.keras.applications, base_model_name)
    base_model = base_model_class(include_top=False, weights='imagenet', input_shape=input_shape)
    base_model.trainable = False

    x = base_model.output
    x = layers.Conv2DTranspose(32, (2, 2), strides=(2, 2), activation='relu')(x)
    x = layers.Conv2D(1, (1, 1), activation='sigmoid')(x)

    model = Model(inputs=base_model.input, outputs=x)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model



## **6. Train Models and Evaluate**

In [ ]:
# Define input shape
input_shape = (None, None, 3)

# List of models to evaluate
base_models = ['ResNet50', 'InceptionV3', 'Xception', 'VGG16', 'VGG19']

# Initialize dictionaries to store training histories and evaluation scores
training_histories = {}
evaluation_scores = {}

# Train and Evaluate each model
for base_model_name in base_models:
    print(f"Training Model: {base_model_name}")
    model = create_segmentation_model(base_model_name, input_shape)

    train_gen = data_generator(train_image_paths, train_mask_paths, batch_size=32, augment=True)
    steps_per_epoch = len(train_image_paths) // 32
    history = model.fit(train_gen, steps_per_epoch=steps_per_epoch, epochs=10)

    training_histories[base_model_name] = history.history

    val_gen = data_generator(val_image_paths, val_mask_paths, batch_size=32, augment=False)
    val_steps = len(val_image_paths) // 32
    val_preds = model.predict(val_gen, steps=val_steps)
    val_true = np.concatenate([mask for _, mask in data_generator(val_image_paths, val_mask_paths, batch_size=32, augment=False)], axis=0)
    f1 = f1_score(val_true.ravel(), val_preds.ravel() > 0.5)

    evaluation_scores[base_model_name] = {'accuracy': history.history['accuracy'][-1], 'f1_score': f1}

# Plotting Training Accuracy
plt.figure(figsize=(10,6))
for model_name, history in training_histories.items():
    plt.plot(history['accuracy'], label=f"{model_name} Training Accuracy")
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(loc='upper left')
plt.title('Training Accuracy Comparison')
plt.show()

# Plotting Evaluation Metrics
models_names = list(evaluation_scores.keys())
accuracies = [score['accuracy'] for score in evaluation_scores.values()]
f1_scores = [score['f1_score'] for score in evaluation_scores.values()]

barWidth = 0.3
r1 = np.arange(len(models_names))
r2 = [x + barWidth for x in r1]

plt.figure(figsize=(10,6))
plt.bar(r1, accuracies, width=barWidth, label='Accuracy')
plt.bar(r2, f1_scores, width=barWidth, label='F1 Score')
plt.xlabel('Models', fontweight='bold')
plt.xticks([r + barWidth / 2 for r in range(len(models_names))], models_names)
plt.legend(loc='upper left')
plt.title('Model Evaluation Comparison')
plt.show()

## **7. Inference on New Images**

In [ ]:
# Select the best model based on evaluation scores
best_model_name = max(evaluation_scores, key=lambda k: evaluation_scores[k]['f1_score'])
best_model = create_segmentation_model(best_model_name, input_shape)

# Train the best model on the training set
train_gen = data_generator(train_image_paths, train_mask_paths, batch_size=32, augment=True)
steps_per_epoch = len(train_image_paths) // 32
best_model.fit(train_gen, steps_per_epoch=steps_per_epoch, epochs=10)

# Define the Inference function
def infer(model, image_path):
    image = cv2.imread(image_path)
    image = image / 255.0
    input_array = np.expand_dims(image, axis=0)
    mask_pred = model.predict(input_array)
    return mask_pred[0]

# List of new image paths for inference
new_image_paths = ['path_to_new_image1', 'path_to_new_image2', ...]  # Replace with actual paths

# Inference and Plot Results
for new_image_path in new_image_paths:
    predicted_mask = infer(best_model, new_image_path)
    plt.figure(figsize=(10,5))
    plt.subplot(1,2,1)
    plt.imshow(cv2.imread(new_image_path))
    plt.title('Original Image')
    plt.subplot(1,2,2)
    plt.imshow(predicted_mask.squeeze(), cmap='gray')
    plt.title('Predicted Mask')
    plt.show()
